# Page 4: Ecological Water Flow Impact Analysis

This notebook analyzes the impact of **Ecological water flow variable** on:
- YRB available surface water
- Discharge at Lijin station (入海净流量)
- Sediment load at Lijin station (输沙量)

The analysis uses time series visualization with mean ± confidence intervals when multiple scenarios are available.


In [ ]:
# Import required libraries
import sys

sys.path.insert(0, "..")

import polars as pl
import plotly.graph_objects as go
from scripts.query_scenarios import ScenarioQuery

# Initialize query engine
query = ScenarioQuery("../data_parquet")

综合分析，为尽可能减少黄河下游河道的泥沙淤积，下游河道多年平均汛期输沙用水量利津断面应在170亿m3左右。(《黄河流域综合规划》p167)

按照生态保护的基本维持、一
般维护和适宜发展的目标，其相应的生态水量比例应
为河道多年平均径流量的30%、40%和 50%~60%。
若按黄河地表径流量580亿m’计，其相应的生态预留
水量（含冲沙水量）分别 174亿、232亿、290亿~348
亿m’。
《关于黄河“八七”分水方案调整的几点思考》

In [ ]:
# ========== 设置 ==========
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd()))

from scripts.query_scenarios import ScenarioQuery
from scripts.analysis_helpers import sensitivity_test

# ========== 初始化 ==========
query = ScenarioQuery("../data_parquet")

# ========== 敏感性分析 ==========
results = sensitivity_test(
    query,
    vary_param="Ecological water flow variable",
    fixed_params=None,
    time_range=(2020, 2100),
    metric="cv",
)

results

对生态流量占比敏感的就只有可用水资源和水压力

In [ ]:
from scripts.query_scenarios import ScenarioQuery

query = ScenarioQuery("../data_parquet")

filters = {
    "Climate change scenario switch for water yield": 2,
    "Fertility Variation": 1.7,
    "water-saving irrigation efficiency ratio": 0.8,
}
data = query.get_series(
    variables="yrb_available_surface_water",
    filters=filters,
    time_range=(2020, 2100),
    include_params=True,
)
data

In [ ]:
from scripts.viz_helpers import quick_plot

quick_plot(
    query,
    variable="yrb_available_surface_water",
    filters=filters,
    time_range=(2020, 2100),
)

In [ ]:
# 计算生态流量
ecological_water = (
    data["value"]
    * data["Ecological water flow variable"]
    / (1 - data["Ecological water flow variable"])
)
# 174亿、232亿、290亿~348亿
threshold = {
    "critical": 17.4,
    "normal": 23.2,
    "safe": 29.0,
    "great": 34.8,
}

In [ ]:
# 假设 data 是你的 Polars DataFrame
# 计算生态流量
data_with_eco = data.with_columns(
    [
        (
            pl.col("value")
            * pl.col("Ecological water flow variable")
            / (1 - pl.col("Ecological water flow variable"))
        ).alias("ecological_water")
    ]
)

# 按时间聚合（计算多情景统计）
eco_stats = (
    data_with_eco.group_by("time")
    .agg(
        [
            pl.col("ecological_water").mean().alias("mean"),
            pl.col("ecological_water").std().alias("std"),
            pl.col("ecological_water").min().alias("min"),
            pl.col("ecological_water").max().alias("max"),
        ]
    )
    .with_columns(
        [
            (pl.col("mean") - 1.96 * pl.col("std")).alias("ci_lower"),
            (pl.col("mean") + 1.96 * pl.col("std")).alias("ci_upper"),
        ]
    )
    .sort("time")
)

# 转换为 pandas
eco_pd = eco_stats.to_pandas()

# 创建图表
fig = go.Figure()

# 最小-最大范围（浅色）
fig.add_trace(
    go.Scatter(
        x=eco_pd["time"],
        y=eco_pd["max"],
        fill=None,
        mode="lines",
        line=dict(color="#2E86AB", width=0),
        showlegend=False,
        hoverinfo="skip",
    )
)

fig.add_trace(
    go.Scatter(
        x=eco_pd["time"],
        y=eco_pd["min"],
        fill="tonexty",
        mode="lines",
        line=dict(color="#2E86AB", width=0),
        fillcolor="rgba(46, 134, 171, 0.1)",
        name="Min-Max Range",
    )
)

# 置信区间（中色）
fig.add_trace(
    go.Scatter(
        x=eco_pd["time"],
        y=eco_pd["ci_upper"],
        fill=None,
        mode="lines",
        line=dict(color="#2E86AB", width=0),
        showlegend=False,
        hoverinfo="skip",
    )
)

fig.add_trace(
    go.Scatter(
        x=eco_pd["time"],
        y=eco_pd["ci_lower"],
        fill="tonexty",
        mode="lines",
        line=dict(color="#2E86AB", width=0),
        fillcolor="rgba(46, 134, 171, 0.3)",
        name="95% CI",
    )
)

# 均值线（粗实线）
fig.add_trace(
    go.Scatter(
        x=eco_pd["time"],
        y=eco_pd["mean"],
        mode="lines",
        line=dict(color="#2E86AB", width=3),
        name="Mean Ecological Water",
    )
)

# 添加阈值线
thresholds = {
    "critical": (17.4, "red", "Critical ($17.4km^3$)"),
    "normal": (23.2, "orange", "Normal ($23.2km^3$)"),
    "safe": (29.0, "yellow", "Safe ($29.0km^3$)"),
    "great": (34.8, "green", "Great ($34.8km^3$)"),
}

for key, (value, color, label) in thresholds.items():
    fig.add_hline(
        y=value,
        line_dash="dash",
        line_color=color,
        line_width=2,
        annotation_text=label,
        annotation_position="right",
        annotation=dict(font_size=11, font_color=color),
    )

# 更新布局
fig.update_layout(
    title=dict(
        text="Ecological Water Flow - Multi-Scenario Analysis",
        font=dict(size=18, family="Arial Black"),
    ),
    xaxis_title="Year",
    yaxis_title="Ecological Water ($km^3$)",
    hovermode="x unified",
    height=700,
    width=1200,
    template="plotly_white",
    legend=dict(
        yanchor="top", y=0.99, xanchor="left", x=0.01, bgcolor="rgba(255,255,255,0.8)"
    ),
)

# 显示
fig.show()

# 保存
fig.write_html("ecological_water_analysis.html")